**Initialization**
* I use these 3 lines of code on top of my each Notebooks because it will help to prevent any problems while reloading and reworking on a same Project or Problem. And the third line of code helps to make visualization within the Notebook.

In [1]:
#@ Initialization:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

**Downloading the Libraries and Dependencies**
* I have downloaded all the Libraries and Dependencies required for this Project in one particular cell.

In [2]:
#@ Downloading the Libraries and Dependencies:
import torch.nn as nn
import numpy as np
import pandas as pd
import re, json, string, os
import collections

from argparse import Namespace
from IPython.display import display 
from collections import Counter
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm_notebook

**Getting the Data**
* I have used Google Colab for this Project so the process of downloading and reading the Data might be different in other platforms. I have used [**Yelp Reviews Dataset**](https://www.kaggle.com/yelp-dataset/yelp-dataset) for this Project. In 2015, Yelp held a contest to predict the Rating of the Restaurants given it's Reviews. Zhang, Zhao, and Lecun simplified the Dataset by converting the Ratings into Sentiments viz. Positive Sentiment for 3 to 4 star Ratings and Negative Sentiment for 1 to 2 star Ratings. The Dataset is splitted into 560,000 Training Samples and 38,000 Testing Samples. 

In [3]:
#@ Getting the Dataset:
args = Namespace(
    raw_train_dataset = "/content/drive/My Drive/Colab Notebooks/YELP Dataset/raw_train.csv",
    raw_test_dataset = "/content/drive/My Drive/Colab Notebooks/YELP Dataset/raw_test.csv",
    proportion_subset_of_train = 0.1,
    train_proportion = 0.7,
    val_proportion = 0.15,
    test_proportion = 0.15,   
    output_munged = "/content/drive/My Drive/Colab Notebooks/YELP Dataset/reviews_with_splits_lite.csv",
    seed = 1337
)

#@ Reading the Raw Dataset:
train_reviews = pd.read_csv(args.raw_train_dataset, header=None, names=["rating", "review"])
train_reviews = train_reviews[~pd.isnull(train_reviews.review)]
test_reviews = pd.read_csv(args.raw_test_dataset, header=None, names=["rating", "review"])
test_reviews = test_reviews[~pd.isnull(test_reviews.review)]

#@ Inspecting the DataFrame:
display(train_reviews.head())
print(" ")
display(test_reviews.head())

,rating,review
0,1,"Unfortunately, the frustration of being Dr. Go..."
1,2,Been going to Dr. Goldberg for over 10 years. ...
2,1,I don't know what Dr. Goldberg was like before...
3,1,I'm writing this review to give you a heads up...
4,2,All the food is great here. But the best thing...


,rating,review
0,1,Ordered a large Mango-Pineapple smoothie. Stay...
1,2,Quite a surprise! \n\nMy wife and I loved thi...
2,1,"First I will say, this is a nice atmosphere an..."
3,2,I was overall pretty impressed by this hotel. ...
4,1,Video link at bottom review. Worst service I h...


**Processing the Dataset**

In [4]:
#@ Creating the Subset of the Reviews Dataset:
by_rating = collections.defaultdict(list)                     # Collections stores the collection of Data.
for _, row in train_reviews.iterrows():
  by_rating[row.rating].append(row.to_dict())

review_subset = []
for _, item_list in sorted(by_rating.items()):
  n_total = len(item_list)
  n_subset = int(args.proportion_subset_of_train * n_total)
  review_subset.extend(item_list[:n_subset])

#@ Creating the DataFrame:
review_subset = pd.DataFrame(review_subset)

#@ Inspecting the DataFrame:
review_subset.head()

,rating,review
0,1,"Unfortunately, the frustration of being Dr. Go..."
1,1,I don't know what Dr. Goldberg was like before...
2,1,I'm writing this review to give you a heads up...
3,1,Wing sauce is like water. Pretty much a lot of...
4,1,Owning a driving range inside the city limits ...


In [5]:
#@ Performing the Basic EDA:
display(train_reviews.rating.value_counts())                   # Inspecting the Number of Ratings.
print(" ")
display(review_subset.rating.value_counts())                   # Inspecting the Number of Ratings.
print(" ")
display(set(review_subset.rating))                             # Unique Ratings in the DataFrame.

2    280000
1    280000
Name: rating, dtype: int64

2    28000
1    28000
Name: rating, dtype: int64

{1, 2}

**Processing the DataFrame**
* Creating Training, Validation and Testing Splits in the DataFrame.

In [6]:
#@ Splitting the Subset by Rating to create New Training, Validation and Testing Splits:
by_rating = collections.defaultdict(list)
for _, row in review_subset.iterrows():
  by_rating[row.rating].append(row.to_dict())

#@ Creating the Split Data:
final_list = []
np.random.seed(args.seed)
for _, item_list in sorted(by_rating.items()):
  np.random.shuffle(item_list)                                     # Shuffling the Data randomly.
  n_total = len(item_list)
  n_train = int(args.train_proportion * n_total)
  n_val = int(args.val_proportion * n_total)
  n_test = int(args.test_proportion * n_total)
  #@ Giving the Data point a split Attribute:
  for item in item_list[:n_train]:
    item["split"] = "train"
  for item in item_list[n_train:n_train+n_val]:
    item["split"] = "val"
  for item in item_list[n_train+n_val:n_train+n_val+n_test]:
    item["split"] = "test" 
  #@ Adding to the Final List:
  final_list.extend(item_list)

#@ Creating the Final DataFrame:
final_reviews = pd.DataFrame(final_list)

#@ Inspecting the Final Result:
display(final_reviews.head())                                     # Inspecting the DataFrame.
print(" ")
display(final_reviews.split.value_counts())                       # Inspecting the Training, Validation and Testing Data.

,rating,review,split
0,1,Terrible place to work for I just heard a stor...,train
1,1,"3 hours, 15 minutes-- total time for an extrem...",train
2,1,My less than stellar review is for service. ...,train
3,1,I'm granting one star because there's no way t...,train
4,1,The food here is mediocre at best. I went afte...,train


train    39200
val       8400
test      8400
Name: split, dtype: int64

**Cleaning the Data**
* I will clean the Data minimally by adding whitespace around Punctuation symbols and Removing Extraneous symbols which are not Punctuations for all the Splits.

In [7]:
#@ Cleaning the Data:
def preprocess_text(text):
  text = text.lower()                                      # Converting into Lowercase.
  text = re.sub(r"([.,!?])", r" \1 ", text)
  text = re.sub(r"[^a-zA-Z.,!?]+", r" ", text)
  return text

#@ Processing the Review Column:
final_reviews["review"] = final_reviews.review.apply(preprocess_text)

#@ Processing the Rating Column:
final_reviews["rating"] = final_reviews.rating.apply({1:"negative", 2:"positive"}.get)

#@ Inspecting the DataFrame:
final_reviews.head(7)

,rating,review,split
0,negative,terrible place to work for i just heard a stor...,train
1,negative,"hours , minutes total time for an extremely s...",train
2,negative,my less than stellar review is for service . w...,train
3,negative,i m granting one star because there s no way t...,train
4,negative,the food here is mediocre at best . i went aft...,train
5,negative,n n nwe looked at our entertainment book for ...,train
6,negative,i had an appointment that was made months in a...,train


In [8]:
#@ Preparing the Data:
final_reviews.to_csv(args.output_munged, index=False)

**PyTorch Dataset Class**
* PyTorch provides an abstraction for the Dataset by providing a Dataset Class. The Dataset Class is an abstract Operator. When using PyTorch with a new Dataset it is necessary to sub class the Dataset Class and Implement the __getitem__ and __len__ methods.

In [9]:
#@ PyTorch Dataset Class:
class ReviewDataset(Dataset):
  def __init__(self, review_df, vectorizer):
    """
    Args: review_df(pandas.DataFrame): The dataset.
        : vectorizer(ReviewVectorizer): Vector instantiated from dataset.
    """
    self.review_df = review_df
    self._vectorizer = vectorizer

    self.train_df = self.review_df[self.review_df.split == "train"]
    self.train_size = len(self.train_df)

    self.val_df = self.review_df[self.review_df.split == "val"]
    self.validation_size = len(self.val_df)

    self.test_df = self.review_df[self.review_df.split == "test"]
    self.test_size = len(self.test_df)

    self._lookup_dict = {"train": (self.train_df, self.train_size),
                           "val": (self.val_df, self.validation_size),
                           "test": (self.test_df, self.test_size)}
    self.set_split("train")
  
  @classmethod
  def load_dataset_and_make_vectorizer(cls, review_csv):
    """Load dataset and make new vectorizer from scratch.
    Args: review_csv: Location of the dataset.
    Returns: An instance of ReviewDataset.
    """
    review_df = pd.read_csv(review_csv)
    train_review_df = review_df[review_df.split == "train"]
    return cls(review_df, ReviewVectorizer.from_datafram(train_review_df))
  
  @classmethod
  def load_dataset_and_load_vectorizer(cls, review_csv, vectorizer_filepath):
    """Load dataset and the corresponding vectorizer.
    Args: review_csv: Location of the dataset.
        : vectorizer_filepath: Location of the saved vectorizer.
    Returns: An instance of the ReviewDataset.
    """
    review_df = pd.read_csv(review_csv)
    vectorizer = cls.load_vectorizer_only(vectorizer_filepath)
    return cls(review_df, vectorizer)

  @staticmethod
  def load_vectorizer_only(vectorizer_filepath):
    """A static method for loading the vectorizer from file.
    Args: vectorizer_filepath: Location of serialized vectorizer.
    Returns: An instance of ReviewVectorizer.
    """
    with open(vectorizer_filepath, "w") as fp:
      json.dump(self._vectorizer.to_serializable(), fp)
  
  def get_vectorizer(self):
    """Returns the Vectorizer.
    """
    return self._vectorizer
  
  def set_split(self, split="train"):
    """Splits the dataset using a column in the DataFrame.
    Args: split(str): One of "train", "val" or "test"
    """
    self._target_split = split
    self._target_df, self._target_size = self._lookup_dict[split]
  
  def __len__(self):
    return self._target_size
  
  def __getitem__(self, index):
    """Primary entry point of PyTorch Datasets.
    Args: index: Index of the Datapoint.
    Returns: A dictionary holding the Data point features and labels.
    """
    row = self._target_df.iloc[index]
    review_vector = self._vectorizer.vectorize(row.review)
    rating_index = self._vectorizer.rating_vocab.lookup_token(row.rating)
    return {"x_data": review_vector,
            "y_target": rating_index}
  
  def get_num_batches(self, batch_size):
    """Given a batch size, return the number of batches in the Dataset.
    Args: batch_size(int)
    Returns: Number of batches in the Dataset.
    """
    return len(self) // batch_size

**The Vocabulary Class**
* The Vocabulary Class not only manages the Bijection i.e Allowing user to add new Tokens and have the Index auto increment but also handles the special token called UNK which stands for Unknown. By using the UNK Token, It will be easy to handle Tokens at Test time that were never seen in Training Instance.

In [10]:
#@ The Vocabulary Class:
class Vocabulary(object):
  """Class to process Text and and extract Vocabulary for mapping.
  """
  def __init__(self, token_to_idx=None, add_unk=True, unk_token="<UNK>"):
    """
    Args: token_to_idx(dict): Pre existing map of Tokens to Index.
        : add_unk(bool): A flag indicating whether to add UNK Token.
        : unk_token(string): The UNK Token to add in Vocabulary.
    """
    if token_to_idx is None:
      token_to_idx = {}
    self.token_to_idx = token_to_idx
    self._idx_to_token = {idx:token for token,idx in self._idx_to_token.items()}
    self._add_unk = add_unk
    self._unk_token = unk_token
    self.unk_index = -1
    if add_unk:
      self.unk_index = self.add_token(unk_token)
  
  def to_serializable(self):
    """Returns a dictionary that can be serialized.
    """
    return {"token_to_idx": self._token_to_index,
            "add_unk": self._add_unk,
            "unk_token": self._unk_token}
  
  @classmethod
  def from_serializable(cls, contents):
    """Instantiate the Vocabulary from the serialized Dictionary.
    """
    return cls(**contents)
  
  def add_token(self, token):
    """Update the mapping dictionary based on the Tokens.
    Args: token: The item to add into the Vocabulary.
    Returns: index: Integer corresponding to the Token.
    """
    if token in self._token_to_idx:
      index = self._token_to_idx[token]
    else:
      index = len(self._token_to_idx)
      self._token_to_idx[token] = index
      self._idx_to_token[index] = token
    return index
  
  def add_many(self, tokens):
    """Add a list of Tokens into Vocabulary.
    Args: tokens(list): A list of string Tokens.
    Returns: indices(list): A list of indices correspoinding to the Tokens.
    """
    return [self.add_token(token) for token in tokens]
  
  def lookup_token(self, token):
    """Retrieve the Index associated with the Token.
    Args: token(str): The Token to lookup.
    Returns: index(int): The Index correspoinding to the Token.
    """
    if self.unk_index >= 0:
      return self._token_to_idx.get(token, self.unk_index)
    else:
      return self._token_to_idx[token]
  
  def lookup_index(self, index):
    """Return the Token associated with the Index.
    Args: index(int): The Index to lookup.
    Returns: token(str): The Token correspoinding to the Index.
    """
    if index not in self._idx_to_token:
      raise KeyError("the index (%d) is not in the Vocabulary" % index)
    return self._idx_to_token[index]
  
  def __str__(self):
    return "<Vocabulary(size=%d)>" % len(self)
  
  def __len__(self):
    return len(self._token_to_idx)

**Vectorizer Class**
* The second stage of going from a Text Dataset to a vectorized minibatch is to iterate through the Tokens of an Input Data Point and convert each Token to its Integer form. The result of this iteration should be a Vector. Because this Vector will be combined with Vectors from other Data points, there is Constraint that the Vectors produced by the Vectorizer should always have the same length.

In [11]:
#@ Vectorizer Class:
class ReviewVectorizer(object):
  """The Vectorizer coordinates the Vocabularies and puts them to use.
  """
  def __init__(self, review_vocab, rating_vocab):
    """
    Args: review_vocab: Maps words to Integers.
        : rating_vocab: Maps class labels to Integers.
    """
    self.review_vocab = review_vocab
    self.rating_vocab = rating_vocab

  def vectorize(self, review):
    """Create a collasped Onehot Vector for the review.
    Args: review: The review
    Returns: one_hot: The collapsed one hot Encoding.
    """
    one_hot = np.zeros(len(self.review_vocab), dtype=np.float32)
    for token in review.split(" "):
      if token not in string.punctuation:
        one_hot[self.review_vocab.lookup_token(token)] = 1
    return one_hot

  @classmethod
  def from_dataframe(cls, review_df, cutoff=25):
    """Instantiate the Vectorizer from DataFrame.
    Args: review_df(DataFrame): The review Dataset.
        :  cufoff(int): Parameter for frequency based Filtering.
    Returns: An instance of the ReviewVectorizer.
    """
    review_vocab = Vocabulary(add_unk=True)
    rating_vocab = Vocabulary(add_unk=False)
    #@ Adding Ratings:
    for rating in sorted(set(review_df.rating)):
      rating_vocab.add_token(rating)
    #@ Adding Topwords if count > provided count:
    word_counts = Counter()
    for review in review_df.review:
      for word in review.split(" "):
        if word not in string.punctuation:
          word_counts[word] += 1
    for word, count in word_counts.items():
      if count > cutoff:
        review_vocab.add_token(word)
    return cls(review_vocab, rating_vocab)

  @classmethod
  def from_serializable(cls, contents):
    """Instantiating the ReviewVectorizer from a serializable dictionary.
    Args: contents: The serializable dictionary.
    Returns: An instance of ReviewVectorizer Class.
    """
    review_vocab = Vocabulary.from_serializable(contents["review_vocab"])
    rating_vocab = Vocabulary.from_serializable(contents["rating_vocab"])
    return cls(review_vocab=review_vocab, rating_vocab=rating_vocab)
  
  def to_serializable(self):
    """Create serializable dictionary for Caching.
    Returns: contents(dict): The Serializable Dictionary.
    """
    return {"review_vocab": self.review_vocab.to_serializable(),
            "rating_vocab": self.rating_vocab.to_serializable()}

**DataLoader**
* The Final step of Text to Vectorized minibatch pipeline is to actually group the Vectorized Datapoints. Because grouping into mini batches is a viatal part of Training the Neural Networks, PyTorch provides a built in class called DataLoader for coordinating the Process.

In [12]:
#@ DataLoader:
def generate_batches(dataset, batch_size, shuffle=True, 
                     drop_last=True, device="gpu"):
  """A generator function which wraps the PyTorch DataLoader. 
  """
  dataloader = DataLoader(dataset=dataset, batch_size=batch_size, 
                          shuffle=shuffle, drop_last=drop_last)
  for data_dict in dataloader:
    out_data_dict = {}
    for name, tensor in data_dict.items():
      out_data_dict[name] = data_dict[name].to(device)
    yield out_data_dict